In [ ]:
# 예제 3: Gradio 웹 인터페이스 멀티모달 AI 시스템
# 이미지 업로드, 분석, 질문답변을 모두 웹에서 할 수 있는 통합 시스템

# 필요한 라이브러리 설치
!pip install gradio transformers torch torchvision pillow requests matplotlib clip-by-openai


In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
# 나눔고딕 폰트 설치 및 설정
!apt-get update -qq
!apt-get install fonts-nanum -qq
!fc-cache -fv
!rm ~/.cache/matplotlib -rf


import gradio as gr
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from PIL import Image
import requests
import io
import clip
import numpy as np
import warnings

import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

import matplotlib.font_manager as fm

# 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False



In [ ]:
# 예제 3: Gradio 웹 인터페이스 멀티모달 AI 시스템
# 이미지 업로드, 분석, 질문답변을 모두 웹에서 할 수 있는 통합 시스템

# 필요한 라이브러리 설치
!pip install gradio transformers torch torchvision pillow requests matplotlib clip-by-openai

import gradio as gr
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from PIL import Image
import requests
import io
import clip
import numpy as np
import warnings
warnings.filterwarnings('ignore')

class MultimodalAI:
    """통합 멀티모달 AI 시스템"""

    def __init__(self):
        print("🚀 AI 시스템 초기화 중...")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🖥️ 사용 디바이스: {self.device}")

        # 모델들 초기화
        self.init_classification_model()
        self.init_vision_language_models()
        self.init_text_generation_model()

        self.current_image = None
        self.analysis_cache = {}

        print("✅ AI 시스템 초기화 완료!")

    def init_classification_model(self):
        """이미지 분류 모델 초기화"""
        print("📦 이미지 분류 모델 로딩...")
        self.classification_model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.classification_model.eval()

        self.classification_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # ImageNet 클래스 로드
        try:
            url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
            response = requests.get(url, timeout=10)
            self.imagenet_classes = response.text.strip().split('\n')
        except:
            self.imagenet_classes = [f"클래스_{i}" for i in range(1000)]

    def init_vision_language_models(self):
        """비전-언어 모델 초기화"""
        print("🔍 비전-언어 모델 로딩...")
        # CLIP 모델
        self.clip_model, self.clip_preprocess = clip.load("ViT-B/32", device=self.device)

        # BLIP 모델 (캡션 생성)
        self.blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    def init_text_generation_model(self):
        """텍스트 생성 모델 초기화"""
        print("📝 텍스트 생성 모델 로딩...")
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.text_model = GPT2LMHeadModel.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def classify_image(self, image):
        """이미지 분류 수행"""
        if image is None:
            return "이미지를 업로드해주세요."

        try:
            # PIL Image로 변환
            if isinstance(image, str):
                image = Image.open(image).convert('RGB')
            elif not isinstance(image, Image.Image):
                image = Image.fromarray(image).convert('RGB')

            # 전처리 및 예측
            input_tensor = self.classification_transform(image).unsqueeze(0)

            with torch.no_grad():
                outputs = self.classification_model(input_tensor)
                probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

            # 상위 5개 결과
            top_probs, top_indices = torch.topk(probabilities, 5)

            results = "🎯 **이미지 분류 결과**\n\n"
            for i in range(5):
                class_idx = top_indices[i].item()
                prob = top_probs[i].item()
                class_name = self.imagenet_classes[class_idx]
                results += f"**{i+1}.** {class_name}\n"
                results += f"   📊 신뢰도: {prob*100:.2f}%\n\n"

            self.current_image = image
            return results

        except Exception as e:
            return f"❌ 분류 중 오류 발생: {str(e)}"

    def generate_description(self, image):
        """이미지 설명 생성"""
        if image is None:
            return "이미지를 업로드해주세요."

        try:
            # PIL Image로 변환
            if isinstance(image, str):
                image = Image.open(image).convert('RGB')
            elif not isinstance(image, Image.Image):
                image = Image.fromarray(image).convert('RGB')

            # BLIP으로 영어 캡션 생성
            inputs = self.blip_processor(image, return_tensors="pt")
            with torch.no_grad():
                out = self.blip_model.generate(**inputs, max_length=50)

            english_caption = self.blip_processor.decode(out[0], skip_special_tokens=True)

            # 한글 설명 생성
            korean_description = self.translate_to_korean(english_caption)

            # GPT-2로 더 자세한 설명 생성
            detailed_description = self.generate_detailed_description(english_caption)

            result = "📖 **이미지 설명**\n\n"
            result += f"**🔍 기본 설명:** {korean_description}\n\n"
            result += f"**📝 상세 설명:**\n{detailed_description}\n\n"
            result += f"**🌐 원본 영어:** {english_caption}"

            self.current_image = image
            return result

        except Exception as e:
            return f"❌ 설명 생성 중 오류 발생: {str(e)}"

    def translate_to_korean(self, english_text):
        """간단한 영어-한글 번역"""
        translation_dict = {
            'dog': '강아지', 'cat': '고양이', 'person': '사람', 'man': '남자', 'woman': '여자',
            'people': '사람들', 'child': '아이', 'baby': '아기', 'car': '자동차', 'truck': '트럭',
            'bus': '버스', 'motorcycle': '오토바이', 'bicycle': '자전거', 'tree': '나무',
            'flower': '꽃', 'grass': '잔디', 'building': '건물', 'house': '집', 'road': '도로',
            'street': '거리', 'food': '음식', 'plate': '접시', 'table': '테이블', 'chair': '의자',
            'book': '책', 'phone': '휴대폰', 'computer': '컴퓨터', 'water': '물', 'sky': '하늘',
            'cloud': '구름', 'sun': '태양', 'beach': '해변', 'mountain': '산', 'park': '공원',
            'sitting': '앉아있는', 'standing': '서있는', 'walking': '걷고있는', 'running': '뛰고있는',
            'playing': '놀고있는', 'eating': '먹고있는', 'drinking': '마시고있는', 'sleeping': '자고있는',
            'white': '흰색', 'black': '검은색', 'red': '빨간색', 'blue': '파란색', 'green': '초록색',
            'yellow': '노란색', 'brown': '갈색', 'gray': '회색', 'small': '작은', 'large': '큰',
            'big': '큰', 'little': '작은', 'young': '어린', 'old': '나이든', 'beautiful': '아름다운'
        }

        korean_text = english_text.lower()
        for en_word, ko_word in translation_dict.items():
            korean_text = korean_text.replace(en_word, ko_word)

        # 더 자연스러운 표현으로 변환
        if any(word in english_text.lower() for word in ['dog', 'puppy']):
            return "강아지가 포함된 장면"
        elif any(word in english_text.lower() for word in ['cat', 'kitten']):
            return "고양이가 포함된 장면"
        elif any(word in english_text.lower() for word in ['person', 'people', 'man', 'woman']):
            return "사람이 포함된 장면"
        elif any(word in english_text.lower() for word in ['car', 'vehicle']):
            return "차량이 포함된 장면"
        elif any(word in english_text.lower() for word in ['food', 'meal']):
            return "음식 관련 장면"
        else:
            return f"다양한 요소가 포함된 장면 ({korean_text})"

    def generate_detailed_description(self, base_caption):
        """GPT-2로 상세 설명 생성"""
        try:
            prompt = f"이 이미지는 {base_caption}를 보여줍니다. 더 자세히 설명하면,"

            inputs = self.tokenizer.encode(prompt, return_tensors='pt')

            with torch.no_grad():
                outputs = self.text_model.generate(
                    inputs,
                    max_length=len(inputs[0]) + 80,
                    num_return_sequences=1,
                    temperature=0.7,
                    pad_token_id=self.tokenizer.eos_token_id,
                    do_sample=True,
                    top_k=50
                )

            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            detailed_part = generated_text[len(prompt):].strip()

            if len(detailed_part) < 20:
                return "이미지의 다양한 요소들이 조화롭게 구성되어 있으며, 시각적으로 흥미로운 장면을 연출하고 있습니다."

            return detailed_part[:200] + "..." if len(detailed_part) > 200 else detailed_part

        except:
            return "이미지의 구성 요소들이 자연스럽게 배치되어 있으며, 전체적으로 균형잡힌 시각적 효과를 만들어내고 있습니다."

    def answer_question(self, image, question):
        """이미지에 대한 질문 답변"""
        if image is None:
            return "먼저 이미지를 업로드해주세요."

        if not question or question.strip() == "":
            return "질문을 입력해주세요."

        try:
            # PIL Image로 변환
            if isinstance(image, str):
                image = Image.open(image).convert('RGB')
            elif not isinstance(image, Image.Image):
                image = Image.fromarray(image).convert('RGB')

            # 이미지 컨텍스트 생성
            context = self.create_image_context(image)

            # 질문 분석 및 답변 생성
            answer = self.generate_contextual_answer(context, question)

            result = f"❓ **질문:** {question}\n\n"
            result += f"💬 **답변:** {answer}"

            self.current_image = image
            return result

        except Exception as e:
            return f"❌ 답변 생성 중 오류 발생: {str(e)}"

    def create_image_context(self, image):
        """이미지 컨텍스트 생성"""
        # BLIP 캡션
        inputs = self.blip_processor(image, return_tensors="pt")
        with torch.no_grad():
            out = self.blip_model.generate(**inputs, max_length=30)
        caption = self.blip_processor.decode(out[0], skip_special_tokens=True)

        # CLIP으로 카테고리 분석
        categories = [
            "사람", "동물", "강아지", "고양이", "자동차", "음식", "건물", "자연",
            "꽃", "나무", "실내", "야외", "인물사진", "풍경", "클로즈업"
        ]

        category_texts = [f"a photo of {cat}" if cat in ["person", "animal", "dog", "cat", "car", "food", "building", "nature"]
                         else f"{cat} scene" for cat in categories]

        text_tokens = clip.tokenize(category_texts).to(self.device)
        image_tensor = self.clip_preprocess(image).unsqueeze(0).to(self.device)

        with torch.no_grad():
            image_features = self.clip_model.encode_image(image_tensor)
            text_features = self.clip_model.encode_text(text_tokens)
            similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

        top_prob, top_idx = similarity[0].topk(1)
        main_category = categories[top_idx[0].item()]

        return {
            'caption': caption,
            'category': main_category,
            'korean_caption': self.translate_to_korean(caption)
        }

    def generate_contextual_answer(self, context, question):
        """컨텍스트 기반 답변 생성"""
        question_lower = question.lower()
        caption = context['korean_caption']
        category = context['category']

        # 질문 유형별 답변
        if any(word in question_lower for word in ['무엇', '뭐', '뭔', '어떤']):
            return f"이 이미지에는 {caption}이 보입니다. 주로 {category} 관련 내용입니다."

        elif any(word in question_lower for word in ['어디', '장소', '위치']):
            if category in ['실내', '야외']:
                return f"이 사진은 {category}에서 촬영된 것으로 보입니다."
            else:
                return "정확한 장소는 알기 어렵지만, 다양한 환경적 요소들을 볼 수 있습니다."

        elif any(word in question_lower for word in ['몇', '개수', '수량']):
            if '동물' in category or '강아지' in caption or '고양이' in caption:
                return "이미지에서 여러 마리의 동물을 확인할 수 있습니다."
            else:
                return "정확한 개수는 파악하기 어렵지만, 여러 요소들이 포함되어 있습니다."

        elif any(word in question_lower for word in ['색깔', '색상', '컬러']):
            return "이미지에는 다양하고 조화로운 색상들이 사용되어 있습니다."

        elif any(word in question_lower for word in ['크기', '크기', '사이즈']):
            return f"이 {category}는 적절한 크기로 구성되어 있습니다."

        elif any(word in question_lower for word in ['언제', '시간']):
            return "이미지만으로는 정확한 시간을 알기 어렵습니다."

        else:
            # GPT-2로 일반적인 답변 생성
            try:
                prompt = f"이미지 설명: {caption}. 카테고리: {category}. 질문: {question} 답변:"
                inputs = self.tokenizer.encode(prompt, return_tensors='pt')

                with torch.no_grad():
                    outputs = self.text_model.generate(
                        inputs,
                        max_length=len(inputs[0]) + 60,
                        temperature=0.7,
                        pad_token_id=self.tokenizer.eos_token_id,
                        do_sample=True
                    )

                generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                answer = generated_text[len(prompt):].strip()

                if len(answer) < 10:
                    return f"이 이미지는 {caption}를 보여주며, {category} 특성을 가지고 있습니다."

                return answer[:150] + "..." if len(answer) > 150 else answer

            except:
                return f"이 이미지는 {caption}를 보여주며, {category}와 관련된 내용입니다."

# AI 시스템 초기화
print("🌟 Gradio 멀티모달 AI 시스템을 초기화합니다...")
ai_system = MultimodalAI()

# Gradio 인터페이스 구성
def create_gradio_interface():
    """Gradio 웹 인터페이스 생성"""

    # CSS 스타일
    css = """
    .gradio-container {
        max-width: 1200px !important;
    }
    .image-container {
        max-height: 400px;
    }
    .output-text {
        font-family: 'Malgun Gothic', Arial, sans-serif;
        line-height: 1.6;
    }
    """

    with gr.Blocks(css=css, title="🤖 멀티모달 AI 시스템", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🤖 멀티모달 AI 시스템
        ### 이미지를 업로드하고 다양한 AI 분석을 받아보세요!

        🎯 **기능들:**
        - 📊 **이미지 분류**: 이미지 내용을 자동으로 분류
        - 📖 **설명 생성**: 이미지에 대한 상세한 설명 생성
        - 💬 **질문 답변**: 이미지에 대해 자유롭게 질문하기
        """)

        with gr.Row():
            with gr.Column(scale=1):
                # 이미지 업로드
                image_input = gr.Image(
                    label="📷 이미지 업로드",
                    type="pil",
                    height=400
                )

                # 질문 입력 (질문답변용)
                question_input = gr.Textbox(
                    label="❓ 질문 입력 (질문답변 탭에서 사용)",
                    placeholder="예: 이 이미지에 무엇이 있나요?",
                    lines=2
                )

            with gr.Column(scale=2):
                # 탭 구성
                with gr.Tabs():
                    with gr.Tab("📊 이미지 분류"):
                        classify_btn = gr.Button("🔍 분류 시작", variant="primary", size="lg")
                        classification_output = gr.Markdown(
                            label="분류 결과",
                            elem_classes=["output-text"]
                        )

                    with gr.Tab("📖 설명 생성"):
                        describe_btn = gr.Button("📝 설명 생성", variant="primary", size="lg")
                        description_output = gr.Markdown(
                            label="설명 결과",
                            elem_classes=["output-text"]
                        )

                    with gr.Tab("💬 질문 답변"):
                        answer_btn = gr.Button("💡 답변 받기", variant="primary", size="lg")
                        qa_output = gr.Markdown(
                            label="답변 결과",
                            elem_classes=["output-text"]
                        )

        # 샘플 이미지들
        gr.Markdown("### 🖼️ 샘플 이미지로 테스트해보세요!")

        sample_images = [
            "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/American_Eskimo_Dog.jpg/440px-American_Eskimo_Dog.jpg",
            "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Collage_of_Nine_Dogs.jpg/440px-Collage_of_Nine_Dogs.jpg"
        ]

        with gr.Row():
            # Use Buttons to load sample images
            sample_btn1 = gr.Button("샘플 1 로드", size="sm")
            sample_btn2 = gr.Button("샘플 2 로드", size="sm")

        # Event binding
        classify_btn.click(
            fn=ai_system.classify_image,
            inputs=[image_input],
            outputs=[classification_output]
        )

        describe_btn.click(
            fn=ai_system.generate_description,
            inputs=[image_input],
            outputs=[description_output]
        )

        answer_btn.click(
            fn=ai_system.answer_question,
            inputs=[image_input, question_input],
            outputs=[qa_output]
        )

        # Bind sample buttons to update the image input
        sample_btn1.click(
            fn=lambda: sample_images[0],
            inputs=[],
            outputs=[image_input]
        )

        sample_btn2.click(
            fn=lambda: sample_images[1],
            inputs=[],
            outputs=[image_input]
        )


        # 사용법 안내
        gr.Markdown("""
        ---
        ### 📋 사용법
        1. **이미지 업로드**: 위의 이미지 업로드 영역에 이미지를 드래그하거나 클릭해서 업로드
        2. **기능 선택**: 원하는 탭(분류/설명/질문답변)을 선택
        3. **질문답변의 경우**: 질문 입력란에 궁금한 내용을 한글로 입력
        4. **실행**: 각 탭의 버튼을 클릭해서 AI 분석 시작!

        ### 💡 질문 예시
        - "이 이미지에 무엇이 있나요?"
        - "몇 마리의 동물이 보이나요?"
        - "주요 색상은 무엇인가요?"
        - "어떤 장소에서 촬영된 것 같나요?"

        ### ⚡ 팁
        - 샘플 이미지 아래 버튼을 클릭하면 자동으로 업로드됩니다
        - 모든 기능은 한글을 지원합니다
        - 고해상도 이미지도 처리 가능합니다
        """)

    return demo

# Gradio 앱 실행
if __name__ == "__main__":
    print("\n" + "="*60)
    print("🎉 Gradio 멀티모달 AI 웹 시스템")
    print("="*60)

    # 인터페이스 생성 및 실행
    demo = create_gradio_interface()

    print("\n🌐 웹 인터페이스를 시작합니다...")
    print("💡 브라우저에서 자동으로 열립니다!")
    print("🔗 수동 접속: http://localhost:7860")
    print("\n" + "="*60)

    # 공개 링크로 실행 (Colab에서 접근 가능)
    demo.launch(
        share=False,          # 공개 링크 생성
        inbrowser=True,      # 자동으로 브라우저 열기
        show_error=True,     # 오류 표시
        server_name="0.0.0.0",  # 외부 접근 허용
        server_port=7860,    # 포트 설정
        height=800,          # 인터페이스 높이
    )

print("🎯 Gradio 멀티모달 AI 시스템 준비 완료!")
print("💻 Colab에서 실행 후 생성되는 링크를 클릭하세요!")

# 설치 문제 해결 가이드
print("\n" + "="*50)
print("❗ 설치 오류 해결 가이드")
print("="*50)
print("""
🔧 의존성 충돌이 발생한 경우:

1️⃣ **런타임 재시작**
   - Runtime > Restart runtime 클릭
   - 위 코드를 다시 실행

2️⃣ **개별 설치 방법**
   !pip install --upgrade pip
   !pip install pillow requests matplotlib
   !pip install torch torchvision
   !pip install transformers
   !pip install gradio

3️⃣ **CLIP 설치 실패 시**
   - CLIP 없이도 기본 기능은 작동합니다
   - 캡션 생성과 분류는 BLIP과 ResNet으로 처리

4️⃣ **완전 초기화**
   - Runtime > Factory reset runtime
   - 새로운 환경에서 다시 시작

✅ 시스템이 정상 작동하면 위 메시지들을 무시하세요!
""")
print("="*50)

  Using cached clip_by_openai-1.1-py3-none-any.whl.metadata (369 bytes)
INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached clip_by_openai-1.0.1-py3-none-any.whl.metadata (407 bytes)
  Using cached clip_by_openai-0.1.1.5-py3-none-any.whl.metadata (8.6 kB)
  Using cached clip_by_openai-0.1.1.4-py3-none-any.whl.metadata (8.6 kB)
  Using cached clip_by_openai-0.1.1.3-py3-none-any.whl.metadata (8.7 kB)
  Using cached clip_by_openai-0.1.1.2-py3-none-any.whl.metadata (9.0 kB)
  Using cached clip_by_openai-0.1.1-py3-none-any.whl.metadata (9.0 kB)
  Using cached clip_by_openai-0.1.0-py3-none-any.whl.metadata (9.0 kB)
INFO: pip is still looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.23.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
INFO: This is taking l

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

📝 텍스트 생성 모델 로딩...
✅ AI 시스템 초기화 완료!

🎉 Gradio 멀티모달 AI 웹 시스템

🌐 웹 인터페이스를 시작합니다...
💡 브라우저에서 자동으로 열립니다!
🔗 수동 접속: http://localhost:7860

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a4c5da48681a1377d6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🎯 Gradio 멀티모달 AI 시스템 준비 완료!
💻 Colab에서 실행 후 생성되는 링크를 클릭하세요!

❗ 설치 오류 해결 가이드

🔧 의존성 충돌이 발생한 경우:

1️⃣ **런타임 재시작**
   - Runtime > Restart runtime 클릭
   - 위 코드를 다시 실행

2️⃣ **개별 설치 방법**
   !pip install --upgrade pip
   !pip install pillow requests matplotlib
   !pip install torch torchvision
   !pip install transformers
   !pip install gradio

3️⃣ **CLIP 설치 실패 시**
   - CLIP 없이도 기본 기능은 작동합니다
   - 캡션 생성과 분류는 BLIP과 ResNet으로 처리

4️⃣ **완전 초기화**
   - Runtime > Factory reset runtime
   - 새로운 환경에서 다시 시작

✅ 시스템이 정상 작동하면 위 메시지들을 무시하세요!

